# Conversione dei dati OpenCUP in LD

In [7]:
import pandas as pd
import rdflib
import json
from IPython.display import Markdown as md
from IPython.display import JSON

In [8]:
from pybars import Compiler
import urllib.parse

# Funzione helper per evitare di ottenere IRI non valide
def encodeURIComponent(this,value):
    return urllib.parse.quote(value)

def apply_handlebars_template(template, data):
    compiler = Compiler()
    context = {
        'encodeURIComponent': encodeURIComponent
    }
    return compiler.compile(template)(data, helpers=context)


### Dati di OpenCUP

In [9]:
with open("srcdata/data/opencup_candidature_comuni_finanziate.json", 'r') as f:
    opencup_json=json.load(f)

Per i dati di OpenCUP le ontologie di riferimento sono:

* L'ontologia delle organizzazioni https://w3id.org/italia/onto/COV/
* L'ontologia dei progetti https://w3id.org/italia/onto/Project/
    

In [10]:
md("All'interno del file ci sono {} progetti CUP".format(len(opencup_json["opencup"])))

All'interno del file ci sono 558 progetti CUP

Il formato di partenza dei dati di OpenCUP è:

In [11]:
JSON(opencup_json["opencup"][0])

<IPython.core.display.JSON object>

Definiamo quindi un template che:
* identifichi ciascun CUP con un'IRI univoca
* riporti il titolo del progetto
* riporti l'ente finanziatore come collegamento a un'organizzazione esterna. NB: in questo caso viene utilizzata la Partita IVA/Codice Fiscale, in quanto questo è il valore che è riportato in OpenCUP

Notare che abbiamo usato l'iterazione all'interno del template e il costrutto `#unless` che permette di generare un JSON-LD valido.

In [12]:
opencup_template="""{
    "@context": {
    "COV": "https://w3id.org/italia/onto/COV/",
    "PRJ": "https://w3id.org/italia/onto/Project/"
    },
    "@graph": [
        {{#each .opencup}}
        {
        "@type": "PRJ:PublicInvestmentProject",
        "@id": "https://w3id.org/italia/data/PublicInvestmentProject/{{encodeURIComponent CUP}}",

        "PRJ:uniqueProjectCodeValue": "{{CUP}}",
        "PRJ:projectTitle": "{{DESCRIZIONE_SINTETICA_CUP}}",
        "PRJ:projectTotalCost": {{COSTO_PROGETTO}},
        "PRJ:projectFunder": {
            "@type":"COV:PublicOrganization",
            "@id":"https://w3id.org/italia/data/PublicOrganization/{{encodeURIComponent PIVA_CODFISCALE_SOG_TITOLARE}}",
            "COV:hasCategory": {
                "@type": "COV:PublicOrganizationCategory",
                "@id":"https://w3id.org/italia/data/PublicOrganizationCategory/{{encodeURIComponent CATEGORIA_SOGGETTO}}"
            },
            "COV:taxCode": "{{PIVA_CODFISCALE_SOG_TITOLARE}}"
        }
        }
        {{#unless @last}},{{/unless}}
        {{/each}}
    ]
}"""

In [13]:
# Trasformazione di JSON in JSON-LD

opencup_json_ld = apply_handlebars_template(opencup_template,opencup_json)

In [24]:
with open ("LD/json-ld/opencup_candidature_comuni_finanziate-ld.json", "w+") as f:
    f.write(opencup_json_ld)

In [14]:
print(opencup_json_ld)

{
    "@context": {
    "COV": "https://w3id.org/italia/onto/COV/",
    "PRJ": "https://w3id.org/italia/onto/Project/"
    },
    "@graph": [
        {
        "@type": "PRJ:PublicInvestmentProject",
        "@id": "https://w3id.org/italia/data/PublicInvestmentProject/J41F22000670001",

        "PRJ:uniqueProjectCodeValue": "J41F22000670001",
        "PRJ:projectTitle": "ABILITAZIONE AL CLOUD PER LE PA LOCALI COMUNI*VIA PAPA GIOVANNI XXIII, 1*ABILITAZIONE AL CLOUD PER LE PA LOCALI COMUNI",
        "PRJ:projectTotalCost": 121992,
        "PRJ:projectFunder": {
            "@type":"COV:PublicOrganization",
            "@id":"https://w3id.org/italia/data/PublicOrganization/00650920168",
            "COV:hasCategory": {
                "@type": "COV:PublicOrganizationCategory",
                "@id":"https://w3id.org/italia/data/PublicOrganizationCategory/ENTI%20TERRITORIALI%20E%20DI%20AMMINISTRAZIONE%20DEL%20TERRITORIO"
            },
            "COV:taxCode": "00650920168"
        }
   

In [21]:
opencup_g = rdflib.Graph()
opencup_g.parse(data=opencup_json_ld,format="json-ld");

In [22]:
opencup_g.print()

@prefix COV: <https://w3id.org/italia/onto/COV/> .
@prefix PRJ: <https://w3id.org/italia/onto/Project/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://w3id.org/italia/data/PublicInvestmentProject/B11C22000490006> a PRJ:PublicInvestmentProject ;
    PRJ:projectFunder <https://w3id.org/italia/data/PublicOrganization/82005730161> ;
    PRJ:projectTitle "MIGRAZIONE AL CLOUD DEI SERVIZI DIGITALI DELL&#x27;AMMINISTRAZIONE*TERRITORIO COMUNALE*N. 14 SERVIZI DA MIGRARE" ;
    PRJ:projectTotalCost 121992 ;
    PRJ:uniqueProjectCodeValue "B11C22000490006" .

<https://w3id.org/italia/data/PublicInvestmentProject/B11C22000940006> a PRJ:PublicInvestmentProject ;
    PRJ:projectFunder <https://w3id.org/italia/data/PublicOrganization/00271100224> ;
    PRJ:projectTitle "MIGRAZIONE AL CLOUD DEI SERVIZI DIGITALI DELL&#x27;AMMINISTRAZIONE*TERRITORIO COMUNALE*N. 9 SERVIZI DA MIGRARE" ;
    PRJ:projectTotalCost 47427 ;
    PRJ:uniqueProjectCodeValue "B11C22000940006" .

<https://w3id.org/it

In [17]:
query="""
SELECT (COUNT(DISTINCT ?CUP) as ?num_CUP) 
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:uniqueProjectCodeValue ?CUP
}
"""

res=opencup_g.query(query)

for row in res:
    print(row)

(rdflib.term.Literal('538', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


Come si vede, i CUP sono in numero inferiore, in quanto ve ne sono di duplicati all'interno del file.

Esempio di query: qual è il totale di costo dei progetti per ciascun ente?

In [24]:
query="""
SELECT ?funder (SUM(?totalCost) AS ?funderTotalCost)
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:projectFunder ?funder;
    PRJ:projectTotalCost ?totalCost.
}
GROUP BY ?funder
ORDER BY DESC(?funderTotalCost)
"""

res=opencup_g.query(query)

for row in res:
    print(row)

(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00339370272'), rdflib.term.Literal('7127677', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/80015010723'), rdflib.term.Literal('4858680', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00297110389'), rdflib.term.Literal('1692369', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00080270838'), rdflib.term.Literal('1547897', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00229080338'), rdflib.term.Literal('1479027', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/Publi

## Dati di IndicePA

I dati di IndicePA ci occorrono, ad esempio, per collegare le partite IVA/Codici Fiscali agli enti